## Bayesian Personalized Ranking (BPR) Algorithm -- by timestamp

In [8]:
from dataloader import LoadRatingFile_HoldKOut
from MFbpr import MFbpr
import multiprocessing as mp
import os
import pandas as pd
import numpy as np

# ShoppingMall = pd.read_csv('/Users/ito/venv/CG-RecoEng/data/SM-focus.csv', sep=',', engine='c')
Tops_trans_df = pd.read_csv('/Users/ito/venv/CG-RecoEng/data/og_data/Tops_supermarket.csv', 
                            parse_dates=['TransactionDate'], sep=',', engine='c')

In [9]:
print Tops_trans_df.shape
Tops_trans_df.head(2)

(1995491, 13)


,TypeGroup,BUID,BranchID,TransactionDate,CustomerID,CardNo,TicketNumber,SKUCode,Spending,DeptCode,SubDeptCode,QTY,tDate2
0,1,150,4294,2017-02-27 20:12:00,312CB58B-09B6-49CA-9F1D-AEF8D56E3F4E,8035450104,40155510,8851954101211,17.00,1.0,3.0,1,2017-02-27
1,1,150,2865,2017-01-18 12:28:00,35FFAE65-80FA-4A2B-8F05-98ACAE82EF86,8028520371,80176812,8851717901607,100.39,1.0,4.0,2,2017-01-18


In [10]:
# productfile1 = '/Users/ito/venv/CG-RecoEng/data/og_data/ProductMaster.csv'
productfile2 = '/Users/ito/venv/CG-RecoEng/data/og_data/ProductMaster_Tops.csv'
# product1 = pd.read_csv(productfile1, sep=',', engine='c')
product2 = pd.read_csv(productfile2, sep=',', engine='c')

In [11]:
# data = ShoppingMall
data = Tops_trans_df

In [12]:
# Creating the data that follows the timestamp format
# type(data.TransactionDate[0])
data['OrdinalTime'] = data.TransactionDate.map(pd._libs.tslib.Timestamp.toordinal)
data['CustomerCat'] = pd.Categorical(list(data.CustomerID)).codes
data['ItemCat'] = pd.Categorical(list(data.SKUCode)).codes
sub_data = data[['CustomerCat','ItemCat','QTY','OrdinalTime']]
sub_data.shape
sub_data.head()

sub_data = sub_data.sort_values(by=['CustomerCat','ItemCat'])
sub_data.to_csv('Tops_BPR_2_data.csv', index=False,header=False)

Match the ID and the "series number" by zipping them

In [13]:
idx = pd.Categorical(list(data.CustomerID)).codes
itm = pd.Categorical(list(data.SKUCode)).codes

idx_match = zip(data.CustomerID, data['CustomerCat'] )
itm_match = zip(data.SKUCode, data['ItemCat'])

In [14]:
idx_match[:5]

[('312CB58B-09B6-49CA-9F1D-AEF8D56E3F4E', 1785),
 ('35FFAE65-80FA-4A2B-8F05-98ACAE82EF86', 1971),
 ('D2F9C8EA-51C7-4824-B7F5-9ADAAE665EDB', 7656),
 ('0FEF3D2E-DFF0-4CAF-8CB2-471FF2404348', 595),
 ('B1E1C8CF-D484-4141-89A5-D24C570EF70A', 6470)]

## Data Preparation

In [15]:
dataset = 'Tops_BPR_2_data.csv'
splitter = ","
hold_k_out = 1

In [16]:
from dataloader import LoadRatingFile_HoldKOut
train, test, num_user, num_item, num_ratings = LoadRatingFile_HoldKOut(dataset, splitter, hold_k_out)
print("Load data (%s) done." %(dataset))
print("#users: %d, #items: %d, #ratings: %d" %(num_user, num_item, num_ratings))

Load data (Tops_BPR_2_data.csv) done.
#users: 9204, #items: 44057, #ratings: 1995491


Setup the parameters for bpr with multiprocessing.

In [19]:
factors = 10
learning_rate = 0.01
reg = 0.01
init_mean = 0
init_stdev = 0.01
maxIter = 30
num_thread = mp.cpu_count()

In [20]:
bpr = MFbpr(train, test, num_user, num_item, factors, learning_rate, reg, init_mean, init_stdev)

In [5]:
# bpr.build_model(maxIter, num_thread, batch_size=32)
bpr.build_model(maxIter, num_thread, batch_size=1000)

num of user = 9204
Training MF-BPR with: learning_rate=0.01, regularization=0.0100, factors=10, #epoch=30, batch_size=1000.
Iter=0 [63.0 s] HitRatio@100 = 0.008, NDCG@100 = 0.003 [4.2 s]
Iter=1 [61.9 s] HitRatio@100 = 0.042, NDCG@100 = 0.011 [13.1 s]
Iter=2 [59.9 s] HitRatio@100 = 0.068, NDCG@100 = 0.017 [18.5 s]
Iter=3 [60.5 s] HitRatio@100 = 0.075, NDCG@100 = 0.019 [19.5 s]
Iter=4 [61.9 s] HitRatio@100 = 0.076, NDCG@100 = 0.019 [21.8 s]
Iter=5 [59.7 s] HitRatio@100 = 0.075, NDCG@100 = 0.019 [20.5 s]
Iter=6 [59.5 s] HitRatio@100 = 0.075, NDCG@100 = 0.019 [22.8 s]
Iter=7 [59.3 s] HitRatio@100 = 0.075, NDCG@100 = 0.020 [20.2 s]
Iter=8 [58.7 s] HitRatio@100 = 0.074, NDCG@100 = 0.018 [20.9 s]
Iter=9 [61.2 s] HitRatio@100 = 0.074, NDCG@100 = 0.018 [21.0 s]
Iter=10 [62.8 s] HitRatio@100 = 0.075, NDCG@100 = 0.018 [22.1 s]
Iter=11 [59.7 s] HitRatio@100 = 0.074, NDCG@100 = 0.018 [21.7 s]
Iter=12 [62.9 s] HitRatio@100 = 0.075, NDCG@100 = 0.019 [22.9 s]
Iter=13 [65.6 s] HitRatio@100 = 0.075, NDC

In [21]:
bpr.predict(test[0][0],test[0][1])

1.738651862470068

## Appendix
### Efficiency test

In [ ]:
@exeTime
def predict1(bpr, u, i):
    res = np.inner(bpr.U_np[u], bpr.V_np[i])
    return res

@exeTime
def predict2(bpr, u, i):
    res = T.dot(bpr.U[u], bpr.V[i]).eval()
    return res

In [ ]:
def exeTime(func):
    def newFunc(*args, **args2):
        t0 = time.time()
        print "@%s, {%s} start" % (time.strftime("%X", time.localtime()), func.__name__)
        back = func(*args, **args2)
        print "@%s, {%s} end" % (time.strftime("%X", time.localtime()), func.__name__)
        print "@%.3fs taken for {%s}" % (time.time() - t0, func.__name__)
        return back
    return newFunc

In [ ]:
alist = np.linspace(1,100,10000000)

%%cython
def c_add(int a, int b):
    cdef int s = a+b
    return s

def p_add(a,b):
    return a+b

%%cython
def cython_mean(double[:] x):
    cdef double total = 0
    for i in range(len(x)):
        total += x[i]
    return total / len(x)

def python_mean(xs):
    total = 0
    for i in range(len(xs)):
        total += xs[i]
    return total/len(xs)

from numba import jit
@jit
def numba_mean(xs):
    total = 0
    for i in range(len(xs)):
        total += xs[i]
    return total/len(xs)


%timeit python_mean(alist)
%timeit cython_mean(alist)

%timeit np.mean(alist)

%timeit numba_mean(alist)

### Reducing mem for data frame

In [13]:
sub_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1995491 entries, 0 to 1995490
Data columns (total 4 columns):
CustomerCat    int16
ItemCat        int32
QTY            int64
OrdinalTime    int64
dtypes: int16(1), int32(1), int64(2)
memory usage: 41.9 MB


In [14]:
sub_data.QTY = sub_data.QTY.astype(np.int16)
sub_data.OrdinalTime = sub_data.OrdinalTime.astype(np.int32)
sub_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1995491 entries, 0 to 1995490
Data columns (total 4 columns):
CustomerCat    int16
ItemCat        int32
QTY            int16
OrdinalTime    int32
dtypes: int16(2), int32(2)
memory usage: 22.8 MB


/Users/ito/venv/CG-RecoEng/lib/python2.7/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
